In [1]:
!pip install sparqlwrapper

In [2]:
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# SPARQL Query
query = """
SELECT DISTINCT ?film ?filmLabel ?image WHERE {
  ?film wdt:P31 wd:Q11424;
   wdt:P154 ?image.
  FILTER (!CONTAINS(LCASE(STR(?image)), ".svg"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100
"""

def get_results(endpoint_url, query):
    """ Exécute la requête SPARQL et retourne les résultats en JSON """
    try:
        user_agent = "WDQS-example Python/%s.%s" % (
            sys.version_info[0],
            sys.version_info[1],
        )
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    except Exception as e:
        print(f"❌ Erreur lors de la requête SPARQL : {e}")
        return None

# Exécuter la requête
results = get_results(endpoint_url, query)

# Stocker les résultats dans un tableau
if results:
    data = []
    for result in results["results"]["bindings"]:
        film = result["film"]["value"] if "film" in result else "N/A"
        label = result["label"]["value"] if "label" in result else "N/A"
        image = result["image"]["value"] if "image" in result else "N/A"
        data.append((film, label, image))


import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?film ?filmLabel ?image WHERE {
  ?film wdt:P31 wd:Q11424;
   wdt:P154 ?image.
  FILTER (!CONTAINS(LCASE(STR(?image)), ".svg"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["film"]["value"],
            result["filmLabel"]["value"],
            result["image"]["value"],
        )
    )

In [3]:
dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

,ville,pays,image
0,http://www.wikidata.org/entity/Q19866947,Le Cercle : Rings,http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q21028668,After Image,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q13534614,Q13534614,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q827105,Delicatessen,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q126183,Working Girl,http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...
95,http://www.wikidata.org/entity/Q1070554,Stuart Little 2,http://commons.wikimedia.org/wiki/Special:File...
96,http://www.wikidata.org/entity/Q846679,Karaté Kid,http://commons.wikimedia.org/wiki/Special:File...
97,http://www.wikidata.org/entity/Q20538737,Q20538737,http://commons.wikimedia.org/wiki/Special:File...
98,http://www.wikidata.org/entity/Q30602709,Jan Žižka,http://commons.wikimedia.org/wiki/Special:File...


In [4]:
import requests
import shutil
import os

def download_image(url, save_path="images"):
    """
    Télécharge une image depuis une URL et l'enregistre dans le dossier spécifié.
    
    :param url: URL de l'image à télécharger
    :param save_path: Dossier où enregistrer l'image (par défaut "images")
    :return: Chemin du fichier téléchargé si succès, None sinon
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    
    # Requête pour récupérer l'image
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)

    if request.status_code == 200:
        # Crée le dossier s'il n'existe pas
        os.makedirs(save_path, exist_ok=True)

        # Récupère le nom de fichier depuis l'URL
        filename = os.path.basename(url)
        file_path = os.path.join(save_path, filename)

        # Sauvegarde l'image
        with open(file_path, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        
        print(f"✅ Image téléchargée : {file_path}")
        return file_path  # Retourne le chemin de l'image sauvegardée

    print(f"❌ Échec du téléchargement : {url}")
    return None  # Retourne None en cas d'échec



In [5]:

# Télécharger les images et stocker les chemins dans une nouvelle colonne
dataframe["downloaded_image"] = dataframe["image"].apply(lambda url: download_image(url))



✅ Image téléchargée : images\RIngsLogo.png
✅ Image téléchargée : images\After%20Image%202001.jpg
✅ Image téléchargée : images\Salinger%20Documentary%20Logo.jpg
✅ Image téléchargée : images\Delicatessen%20movie%20logo.png
✅ Image téléchargée : images\Working%20Girl.jpg
✅ Image téléchargée : images\The%20Silence%20of%20the%20Lambs.png
✅ Image téléchargée : images\Viridiana%20movie%20black%20logo.png
✅ Image téléchargée : images\Ghost-World.png
✅ Image téléchargée : images\Pink%20Panther%20logo-en.jpg
✅ Image téléchargée : images\The%20SpongeBob%20Movie%20Sponge%20Out%20of%20Water%20logo.png
✅ Image téléchargée : images\Fight%20Club%20logo1.png
✅ Image téléchargée : images\Predator%20logo.jpg
✅ Image téléchargée : images\D9%20Logo.jpg
✅ Image téléchargée : images\The%20Runaways%20logo.jpg
✅ Image téléchargée : images\On%20the%20Waterfront%20%28logo%29.png
✅ Image téléchargée : images\Open%20season%20logo.PNG
✅ Image téléchargée : images\X2x-menunitedlogo.jpg
✅ Image téléchargée : images\T

In [6]:

def count_files_in_folder(folder_path):
    try:
        return len(os.listdir(folder_path))  # Compte tous les fichiers et dossiers
    except FileNotFoundError:
        print("❌ Le dossier n'existe pas.")
        return 0

# Exemple d'utilisation
folder = "images"  # Remplace par ton dossier
count = count_files_in_folder(folder)
print(f"📂 Le dossier '{folder}' contient {count} éléments.")


📂 Le dossier 'images' contient 98 éléments.


In [7]:
pip install pillow exifread


Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS

# Définir le chemin du dossier contenant les images
image_folder = "images/"  # Modifiez ce chemin si nécessaire

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Le dossier '{image_folder}' n'existe pas.")

# Liste des fichiers image dans le dossier
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.tiff', '.bmp', '.png'))]

# Liste pour stocker les données EXIF
exif_data_list = []

# Parcourir chaque image et extraire les métadonnées EXIF
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        
        # Dictionnaire pour stocker les métadonnées de cette image
        image_exif_dict = {'Filename': image_file}
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, f"Tag_{tag}")  # Convertir le tag en nom lisible
                image_exif_dict[tag_name] = value
        
        exif_data_list.append(image_exif_dict)
    except Exception as e:
        print(f"Erreur lors du traitement de {image_file}: {e}")

# Création du DataFrame
df_exif = pd.DataFrame(exif_data_list)

# Afficher le DataFrame
print(df_exif)

# Sauvegarder les métadonnées EXIF dans un fichier CSV
df_exif.to_csv("exif_metadata.csv", index=False)


                                             Filename  ExifOffset  \
0         2001%20A%20Space%20Odyssey%20%28logo%29.png         NaN   
1                    2011-12-14-missionimpossible.jpg         NaN   
2   4%20luni%2C%203%20s%C4%83pt%C4%83m%C3%A2ni%20%...         NaN   
3                   A%20Cinderella%20Story%20logo.png         NaN   
4                                   Absurd%20Logo.png         NaN   
..                                                ...         ...   
92                              Welcome%20Zindagi.jpg       164.0   
93  West%20Side%20Story%20%282021%20film%20logo%20...         NaN   
94                                 Working%20Girl.jpg         NaN   
95      X-Men%20Days%20of%20Future%20Past%20Title.png         NaN   
96                              X2x-menunitedlogo.jpg         NaN   

                       Software  Orientation             DateTime Tag_59932  \
0                           NaN          NaN                  NaN       NaN   
1            

In [9]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS

# Définir le chemin du dossier contenant les images
image_folder = "images/"  # Modifiez ce chemin si nécessaire

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Le dossier '{image_folder}' n'existe pas.")

# Liste des fichiers image dans le dossier
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

# Liste pour stocker les données EXIF
exif_data_list = []

# Parcourir chaque image et extraire les métadonnées EXIF
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        
        # Dictionnaire pour stocker les métadonnées de cette image
        image_exif_dict = {'Filename': image_file}
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, f"Tag_{tag}")  # Convertir le tag en nom lisible
                image_exif_dict[tag_name] = value
        
        exif_data_list.append(image_exif_dict)
    except Exception as e:
        print(f"Erreur lors du traitement de {image_file}: {e}")

# Création du DataFrame
df_exif = pd.DataFrame(exif_data_list)

# Filtrer les lignes en excluant celles ayant une majorité de valeurs NaN
threshold = 0.5  # Seuil : une ligne est exclue si plus de 50% de ses valeurs sont NaN
df_filtered = df_exif.dropna(thresh=int(threshold * df_exif.shape[1]))

# Afficher le DataFrame filtré
print(df_filtered)

# Sauvegarder les métadonnées EXIF filtrées dans un fichier CSV
df_filtered.to_csv("exif_metadata_filtered.csv", index=False)


                                 Filename  ExifOffset  \
8                           MEANGIRLS.jpg       164.0   
11                      Nightmare%206.jpg       146.0   
12  Nightsesseion%20der%20film%20logo.jpg       172.0   
16      Salinger%20Documentary%20Logo.jpg       156.0   
17            Spike%20Jonze%20-%20Her.jpg       156.0   
21              The%20Runaways%20logo.jpg       164.0   
23                  Welcome%20Zindagi.jpg       164.0   

                               Software  Orientation             DateTime  \
8           Adobe Photoshop CS3 Windows          1.0  2011:03:05 22:45:19   
11                           GIMP 2.6.8          1.0  2011:03:15 19:48:58   
12  Adobe Photoshop CC 2015 (Macintosh)          1.0  2016:03:20 07:41:19   
16                 Adobe Photoshop 7.0           1.0  2013:07:26 18:30:39   
17                 Adobe Photoshop 7.0           1.0  2013:08:20 17:38:22   
21          Adobe Photoshop CS2 Windows          1.0  2010:01:15 14:30:18   
23  

In [21]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS

# Définir le chemin du dossier contenant les images
image_folder = "images/"  # Modifiez ce chemin selon votre dossier

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Le dossier '{image_folder}' n'existe pas.")

# Liste des fichiers image dans le dossier
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif'))]

# Liste pour stocker les données EXIF
exif_data_list = []

# Fonction pour convertir des valeurs EXIF en types sérialisables JSON
def convert_exif_value(value):
    if isinstance(value, bytes):
        try:
            return value.decode('utf-8', errors='ignore')
        except Exception:
            return str(value)
    elif isinstance(value, tuple):  # Convertir tuple en liste
        return list(value)
    elif isinstance(value, Image.Exif.IFDRational):  # Convertir IFDRational en float
        return float(value)
    return value

# Fonction pour déterminer l'orientation de l'image
def get_orientation(width, height):
    if width > height:
        return "Paysage"
    elif width < height:
        return "Portrait"
    else:
        return "Carré"

# Extraction des métadonnées de chaque image
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    try:
        img = Image.open(image_path)
        width, height = img.size  # Taille de l'image
        format_image = img.format  # Format de l'image

        # Métadonnées EXIF
        exif_data = img._getexif()
        image_exif_dict = {
            "Nom du fichier": image_file,
            "Taille": f"{width}x{height}",
            "Format": format_image,
            "Orientation": get_orientation(width, height)
        }
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, f"Tag_{tag}")
                image_exif_dict[tag_name] = convert_exif_value(value)
        
        # Ajout au JSON final
        exif_data_list.append(image_exif_dict)

    except Exception as e:
        print(f"Erreur lors du traitement de {image_file}: {e}")

# Sauvegarde des métadonnées dans un fichier JSON
json_output_path = "Projet Theotim x Daniel/exif_metadata.json"

with open(json_output_path, "w", encoding="utf-8") as json_file:
    json.dump(exif_data_list, json_file, indent=4, ensure_ascii=False)

print(f"Fichier JSON sauvegardé sous : {json_output_path}")


Erreur lors du traitement de Blair-Witch-Project-logo.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de D9%20Logo.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de MEANGIRLS.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Meier%20M%C3%BCller-Schrift.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Nightmare%206.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Nightsesseion%20der%20film%20logo.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Salinger%20Documentary%20Logo.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Spike%20Jonze%20-%20Her.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de Subway-Film.jpg: type object 'Exif' has no attribute 'IFDRational'
Erreur lors du traitement de The%20Game%20%28David%20Finc

In [24]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.cluster import MiniBatchKMeans

# 📂 Chemins des fichiers
image_folder = "images"
json_file_path = "Projet Theotim x Daniel/exif_metadata.json"

# 📖 Charger le fichier JSON existant
with open(json_file_path, "r", encoding="utf-8") as file:
    metadata = json.load(file)

# 🔄 Créer un set des images présentes dans le JSON
json_images = {img_data["Nom du fichier"] for img_data in metadata}

# 🔄 Lister les images du dossier qui existent aussi dans le JSON
image_files = [f for f in os.listdir(image_folder) if f in json_images]

# 🔍 Traiter chaque image du dossier SI ELLE EXISTE DANS LE JSON
for img_name in image_files:
    img_path = os.path.join(image_folder, img_name)
    print(f"\n🔍 Analyse de l'image : {img_name}")

    # 📸 Ouvrir l'image et la convertir en tableau numpy
    img = Image.open(img_path).convert("RGB")
    img_array = np.array(img).reshape(-1, 3)  # Transformer en liste de pixels (RGB)

    # 🎯 Appliquer K-Means pour trouver les 5 couleurs dominantes
    cluster_count = 3
    kmeans = MiniBatchKMeans(n_clusters=cluster_count, n_init=10, random_state=0)
    kmeans.fit(img_array)
    # Récupérer les couleurs des clusters
    cluster_centers = kmeans.cluster_centers_.astype(int)
    
    # Obtenir les labels des pixels
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    
    # Trier les clusters par fréquence d'apparition (du plus fréquent au moins fréquent)
    sorted_indices = np.argsort(-counts)  # Trie en ordre décroissant
    
    # Réorganiser les couleurs et les fréquences selon ce tri
    sorted_cluster_centers = cluster_centers[sorted_indices]
    sorted_colors = [tuple(color) for color in sorted_cluster_centers]    
    # Afficher les résultats
    print("Couleurs triées (RGB) :", [tuple(color) for color in sorted_cluster_centers])
   # print("Nombre de pixels par cluster :", sorted_counts)
    print(sorted_colors[0])

    # 🔝 Conserver le format RGB sous forme de tuples
    rgb_colors = [tuple(color) for color in cluster_centers]
    
    # 🔄 Mettre à jour le JSON avec les couleurs
    for img_data in metadata:
        if img_data["Nom du fichier"] == img_name:
            img_data["dominant_colors"] = f"{sorted_colors[0][0]} {sorted_colors[0][1]} {sorted_colors[0][2]}"
            break  # Une fois mis à jour, on passe à l'image suivante

    # 🎨 Afficher la couleur la plus dominante
    print(f"🌟 Couleur principale : {rgb_colors}")

# 💾 Sauvegarder le JSON mis à jour
with open(json_file_path, "w", encoding="utf-8") as file:
    json.dump(metadata, file, indent=4, ensure_ascii=False)

print("✅ Mise à jour du JSON terminée !")



🔍 Analyse de l'image : 2001%20A%20Space%20Odyssey%20%28logo%29.png


C:\Users\squin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Couleurs triées (RGB) : [(1, 1, 1), (250, 250, 250), (123, 123, 123)]
(1, 1, 1)
🌟 Couleur principale : [(250, 250, 250), (1, 1, 1), (123, 123, 123)]

🔍 Analyse de l'image : 2011-12-14-missionimpossible.jpg


C:\Users\squin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Couleurs triées (RGB) : [(3, 3, 3), (67, 67, 70), (165, 165, 168)]
(3, 3, 3)
🌟 Couleur principale : [(3, 3, 3), (165, 165, 168), (67, 67, 70)]

🔍 Analyse de l'image : 4%20luni%2C%203%20s%C4%83pt%C4%83m%C3%A2ni%20%C5%9Fi%202%20zile%20Sigl%C4%83.png


...

Couleurs triées (RGB) : [(4, 1, 1), (153, 39, 60), (88, 40, 47)]
(4, 1, 1)
🌟 Couleur principale : [(4, 1, 1), (153, 39, 60), (88, 40, 47)]

🔍 Analyse de l'image : X-Men%20Days%20of%20Future%20Past%20Title.png


C:\Users\squin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Couleurs triées (RGB) : [(0, 0, 0), (3, 2, 2), (0, 0, 0)]
(0, 0, 0)
🌟 Couleur principale : [(0, 0, 0), (3, 2, 2), (0, 0, 0)]

🔍 Analyse de l'image : X2x-menunitedlogo.jpg


C:\Users\squin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Couleurs triées (RGB) : [(245, 247, 250), (154, 170, 194), (43, 48, 65)]
(245, 247, 250)
🌟 Couleur principale : [(154, 170, 194), (43, 48, 65), (245, 247, 250)]
✅ Mise à jour du JSON terminée !


In [27]:
import os
from PIL import Image
from PIL.ExifTags import TAGS
from PIL import TiffImagePlugin

# Définir le dossier contenant les images
image_folder = "Projet Theotim x Daniel/images"  # Modifiez ce chemin selon votre dossier

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Le dossier '{image_folder}' n'existe pas.")

# Liste des fichiers image JPEG et TIFF dans le dossier
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.tiff'))]

if not image_files:
    print("❌ Aucune image JPEG ou TIFF trouvée dans le dossier.")
    exit()

# Afficher la liste des images disponibles
print("\n📂 Images disponibles :")
for idx, img in enumerate(image_files, 1):
    print(f"{idx}. {img}")

# Demander à l'utilisateur de choisir une image
while True:
    try:
        choice = int(input("\nEntrez le numéro de l'image à modifier : ")) - 1
        if 0 <= choice < len(image_files):
            image_name = image_files[choice]
            break
        else:
            print("❌ Numéro invalide, essayez encore.")
    except ValueError:
        print("❌ Veuillez entrer un nombre.")

# Charger l'image sélectionnée
image_path = os.path.join(image_folder, image_name)
img = Image.open(image_path)

# Demander les tags à l'utilisateur
tags = input("\nEntrez les tags (séparés par des virgules) : ").strip()
if not tags:
    print("❌ Aucun tag fourni, opération annulée.")
    exit()

# Vérifier et récupérer les métadonnées EXIF existantes
if "exif" in img.info:
    exif_dict = img.getexif()
else:
    exif_dict = TiffImagePlugin.ImageFileDirectory_v2()

# Ajouter les tags sous forme de description
exif_dict[270] = tags  # 270 = ImageDescription en EXIF

# Sauvegarde en remplaçant l'image existante
img.save(image_path, exif=exif_dict)
print(f"\n✅ Tags ajoutés avec succès à {image_name} !")



📂 Images disponibles :
1. 2011-12-14-missionimpossible.jpg
2. After%20Image%202001.jpg
3. Blair-Witch-Project-logo.jpg
4. Child%27s%20Play%202.jpg
5. D9%20Logo.jpg
6. Die%20Katze%201988.jpg
7. Lava%20logo.jpg
8. Logo%20Stuart%20Little%202.JPG
9. MEANGIRLS.jpg
10. Meier%20M%C3%BCller-Schrift.jpg
11. Murder%20Mestri.jpg
12. Nightmare%206.jpg
13. Nightsesseion%20der%20film%20logo.jpg
14. Pink%20Panther%20logo-en.jpg
15. Predator%20logo.jpg
16. Rakta%20Charitra%20Logo%20-%20Flickr%20-%20rgvzoomin.jpg
17. Salinger%20Documentary%20Logo.jpg
18. Spike%20Jonze%20-%20Her.jpg
19. Subway-Film.jpg
20. The%20Ghost%20and%20Mrs.%20Muir%20%281947%29%20trailer%20title.jpg
21. The%20Phantom%20of%20the%20Opera%20title%20card.jpg
22. The%20Runaways%20logo.jpg
23. TheRingThing.jpeg
24. Welcome%20Zindagi.jpg
25. Working%20Girl.jpg
26. X2x-menunitedlogo.jpg



Entrez le numéro de l'image à modifier :  19

Entrez les tags (séparés par des virgules) :  Sous-marin,ocean



✅ Tags ajoutés avec succès à Subway-Film.jpg !


In [30]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS
import ace_tools as tools

image_folder = "Projet Theotim x Daniel/images"  # Modifiez ce chemin selon votre dossier

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Le dossier '{image_folder}' n'existe pas.")

# Liste des fichiers image JPEG et TIFF dans le dossier
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.tiff'))]

# Afficher la liste des images disponibles
print("\n📂 Images disponibles :")
for idx, img in enumerate(image_files, 1):
    print(f"{idx}. {img}")
    
# Liste pour stocker les données
data = []

# Extraction des tags de chaque image
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    try:
        img = Image.open(image_path)
        
        # Vérifier si l'image contient des métadonnées EXIF
        exif_data = img.getexif()
        image_tags = ""

        if exif_data:
            image_tags = exif_data.get(270, "")  # 270 = ImageDescription (où les tags sont stockés)

        # Ajouter les données à la liste
        data.append({"Nom du fichier": image_file, "Tags": image_tags})

    except Exception as e:
        print(f"❌ Erreur lors du traitement de {image_file}: {e}")

# Création du DataFrame
df_tags = pd.DataFrame(data)

# Affichage du DataFrame
df


ModuleNotFoundError: No module named 'ace_tools'